In [1]:
import geopandas as gp
import pandas as pd
import numpy as np
import os
import glob
from pprint import pprint
from itertools import islice
import random

# Set the max_columns option to None to display all columns
pd.set_option('display.max_columns', None)

In [2]:
# set common crs for project
# epsg3310: https://epsg.io/3310-1739
# units: meters
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 


In [3]:
economic_df = pd.read_csv('/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/df_foranalysis.csv')
us_counties = gp.read_file("/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/tl_2019_us_county/tl_2019_us_county.shp")  # CA counties

In [4]:
economic_df.head(2)

,region,division,state,county,stname,ctyname,geofips,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,yr_2021,description,linecode,avg_growthrate_5y,avg_growthrate_10y,avg_growthrate_20y
0,4,8,4,1,Arizona,Apache County,4001,NaN,0.027246,0.020066,0.030087,0.030314,0.020613,0.06893,-0.005125,-0.011229,-0.027159,-0.011921,-0.012065,-0.022241,0.003471,0.012141,-0.009875,0.001843,0.000972,-0.011801,0.059831,0.006599,ann_growthrate,40,0.011489,0.002888,0.008535
1,4,8,4,1,Arizona,Apache County,4001,34632.0,35602.000000,36331.000000,37458.000000,38629.000000,39442.000000,42362.00000,42146.000000,41678.000000,40576.000000,40098.000000,39620.000000,38758.000000,38893.000000,39371.000000,38986.000000,39058.000000,39096.000000,38640.000000,41099.000000,41372.000000,jobs_per_100k,30,NaN,NaN,NaN


In [5]:
us_counties.head(2)

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477652222,10690952,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680962890,61582307,+46.2946377,-123.4244583,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."


In [6]:
us_counties['GEOID'] = us_counties['GEOID'].astype(int)

In [7]:
# merge
merged_df = economic_df.merge(us_counties[['GEOID', 'geometry']], left_on='geofips', right_on='GEOID', how='left')

merged_df = merged_df.drop(columns='GEOID')

gdf = gp.GeoDataFrame(merged_df, geometry='geometry')

In [8]:
type(merged_df)

pandas.core.frame.DataFrame

In [9]:
type(gdf)

geopandas.geodataframe.GeoDataFrame

In [10]:
gdf = gdf.rename(columns={'avg_growthrate_5y': 'avg_gr_5y'})
gdf = gdf.rename(columns={'avg_growthrate_10y': 'avg_gr_10y'})
gdf = gdf.rename(columns={'avg_growthrate_20y': 'avg_gr_20y'})

In [11]:
gdf

,region,division,state,county,stname,ctyname,geofips,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,yr_2021,description,linecode,avg_gr_5y,avg_gr_10y,avg_gr_20y,geometry
0,4,8,4,1,Arizona,Apache County,4001,NaN,0.027246,0.020066,0.030087,0.030314,0.020613,0.068930,-0.005125,-0.011229,-0.027159,-0.011921,-0.012065,-0.022241,0.003471,0.012141,-0.009875,0.001843,0.000972,-0.011801,0.059831,0.006599,ann_growthrate,40,0.011489,0.002888,0.008535,"POLYGON ((-109.84327 35.17040, -109.84370 35.1..."
1,4,8,4,1,Arizona,Apache County,4001,34632.0,35602.000000,36331.000000,37458.000000,38629.000000,39442.000000,42362.000000,42146.000000,41678.000000,40576.000000,40098.000000,39620.000000,38758.000000,38893.000000,39371.000000,38986.000000,39058.000000,39096.000000,38640.000000,41099.000000,41372.000000,jobs_per_100k,30,NaN,NaN,NaN,"POLYGON ((-109.84327 35.17040, -109.84370 35.1..."
2,4,8,4,1,Arizona,Apache County,4001,67863.0,67319.000000,68072.000000,68161.000000,68521.000000,69390.000000,69602.000000,69883.000000,71008.000000,71828.000000,72181.000000,72231.000000,72333.000000,71808.000000,71026.000000,71418.000000,71538.000000,71819.000000,71920.000000,65911.000000,65385.000000,population,20,NaN,NaN,NaN,"POLYGON ((-109.84327 35.17040, -109.84370 35.1..."
3,4,8,4,1,Arizona,Apache County,4001,23502.0,23967.000000,24731.000000,25532.000000,26469.000000,27369.000000,29485.000000,29453.000000,29595.000000,29145.000000,28943.000000,28618.000000,28035.000000,27928.000000,27964.000000,27843.000000,27941.000000,28078.000000,27790.000000,27089.000000,27051.000000,total employment,10,NaN,NaN,NaN,"POLYGON ((-109.84327 35.17040, -109.84370 35.1..."
4,4,8,4,3,Arizona,Cochise County,4003,NaN,-0.011890,0.027794,0.000598,0.017390,0.005264,0.020469,-0.020966,-0.019845,-0.016014,-0.017262,-0.014872,-0.013858,0.001048,-0.002811,-0.005579,0.004453,-0.007618,0.000961,0.032896,-0.007004,ann_growthrate,40,0.004737,-0.001238,-0.001342,"POLYGON ((-110.45230 31.74455, -110.45230 31.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,4,8,56,43,Wyoming,Washakie County,56043,5232.0,5148.000000,5157.000000,5189.000000,5297.000000,5370.000000,5597.000000,5750.000000,5664.000000,5522.000000,5603.000000,5530.000000,5425.000000,5389.000000,5394.000000,5250.000000,5302.000000,5254.000000,5092.000000,5079.000000,5174.000000,total employment,10,NaN,NaN,NaN,"POLYGON ((-107.63349 43.64634, -107.63352 43.6..."
1652,4,8,56,45,Wyoming,Weston County,56045,NaN,-0.042811,-0.010397,-0.009876,0.030020,-0.011288,-0.004822,0.018298,-0.064975,0.020801,-0.023504,0.047497,-0.009848,0.041072,-0.008787,-0.014524,0.011636,-0.004648,0.010583,0.008251,0.014664,ann_growthrate,40,0.008097,0.009590,-0.000133,"POLYGON ((-105.07913 43.81301, -105.07913 43.8..."
1653,4,8,56,45,Wyoming,Weston County,56045,58070.0,55686.000000,55113.000000,54574.000000,56263.000000,55635.000000,55368.000000,56400.000000,52959.000000,54084.000000,52842.000000,55477.000000,54936.000000,57289.000000,56790.000000,55977.000000,56636.000000,56374.000000,56977.000000,57451.000000,58306.000000,jobs_per_100k,30,NaN,NaN,NaN,"POLYGON ((-105.07913 43.81301, -105.07913 43.8..."
1654,4,8,56,45,Wyoming,Weston County,56045,6487.0,6578.000000,6610.000000,6646.000000,6594.000000,6717.000000,7033.000000,7133.000000,7266.000000,7198.000000,7142.000000,7075.000000,7132.000000,7134.000000,7202.000000,7228.000000,6962.000000,6895.000000,6880.000000,6818.000000,6766.000000,population,20,NaN,NaN,NaN,"POLYGON ((-105.07913 43.81301, -105.07913 43.8..."


In [12]:

# subset to just the average growth rate data for ease of joining tables in ArcGIS
gdf_avggrowthrate = gdf[gdf['linecode'] == 40].copy()
gdf_avggrowthrate.to_file('/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/geodf_growthrate.shp', driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
gdf.to_file('/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/geodf_foranalysis.shp', driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.
